# Joins

In [0]:
from pyspark.sql.functions import col, max

blob_container = "blobcontainer" # The name of your container created in https://portal.azure.com
storage_account = "w261section05group03" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261section05group04" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "houseofthedragon" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, datediff, when
print("Welcome to the W261 final project HEY") #hey everyone!Hi HELLO good morning

Welcome to the W261 final project HEY


In [0]:
# Inspect the Mount's Final Project folder 
# Please IGNORE dbutils.fs.cp("/mnt/mids-w261/datasets_final_project/stations_data/", "/mnt/mids-w261/datasets_final_project_2022/stations_data/", recurse=True)
data_BASE_DIR = "dbfs:/mnt/mids-w261/datasets_final_project_2022/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/,parquet_airlines_data/,0,1656618287000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/,parquet_airlines_data_1y/,0,1656630272000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0,1656630114000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0,1656630205000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/,parquet_weather_data/,0,1656622074000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_1y/,parquet_weather_data_1y/,0,1656631614000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m/,parquet_weather_data_3m/,0,1656630651000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_6m/,parquet_weather_data_6m/,0,1656631047000
dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/,stations_data/,0,1656713663000


In [0]:
# Set partitions
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.files.minPartitionNum", 1000)

In [0]:
# Create and save the Aiport Dataset with IATA ICAO and Timezones

!pip install -U airportsdata
import airportsdata
import pandas as pd
#get timezone
gadb = airportsdata.load()
df_gadb = pd.DataFrame.from_dict(gadb).transpose()
#get rid of blank iata codes
df_gadb = df_gadb[df_gadb['iata'] != ""]
df_gadb = spark.createDataFrame(df_gadb)

df_airport_timezone = df_gadb.distinct()

# There are 12 airports missing in this dataset present in the whole flights datasets
values = [
  ("KLCK", "LCK", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/New_York"),
  ("KWYS", "WYS", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Denver"),
  ("KSCK", "SCK", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Los_Angeles"),
  ("KPGD", "PGD", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/New_York"),
  ("KVEL", "VEL", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Denver"),
  ("KCNY", "CNY", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Denver"),
  ("KHOB", "HOB", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Denver"),
  ("KESC", "ESC", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Detroit"),
  ("KRIW", "RIW", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Denver"),
  ("KCRQ", "CLD", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Los_Angeles"),
  ("KEFD", "EFD", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Chicago"),
  ("KTKI", "TKI", "x", "x", "x", "x", 0.0, 0.0, 0.0, "America/Chicago")
]

df_missing_airports = spark.createDataFrame(values, df_airport_timezone.columns)

df_airport_timezone = df_airport_timezone.union(df_missing_airports)

df_airport_timezone = df_airport_timezone.withColumnRenamed("name", "airport_name") \
  .withColumnRenamed("city", "airport_city") \
  .withColumnRenamed("subd", "airport_subd") \
  .withColumnRenamed("country", "airport_country") \
  .withColumnRenamed("lat", "airport_lat") \
  .withColumnRenamed("lon", "airport_lon") \
  .withColumnRenamed("tz", "airport_tz")

df_airport_timezone.write.mode("overwrite").parquet(f"{blob_url}/airport_codes_timezone")

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-a9095a0e-7b7e-49d6-9a3e-0ad47f2d4298/bin/python -m pip install --upgrade pip' command.


In [0]:
# Load the Flights data and remove duplicates 
df_airlines = spark.read.parquet(f"{data_BASE_DIR}parquet_airlines_data/")
df_airlines = df_airlines.distinct()
df_airlines.write.mode("overwrite").parquet(f"{blob_url}/df_airlines_distinct")

In [0]:
# Load all 4 datasets for initial joins
df_airlines = spark.read.parquet(f"{blob_url}/df_airlines_distinct")
df_airport_timezone = spark.read.parquet(f"{blob_url}/airport_codes_timezone")
df_weather = spark.read.parquet(f"{data_BASE_DIR}parquet_weather_data/")
df_stations = spark.read.parquet(f"{data_BASE_DIR}stations_data/*")

In [0]:
df_airlines.count() # 3m 1403471 1y 7422037 all 42430592 3.66

Out[171]: 42430592

In [0]:
# Drop all columns but neighbor id and icao
df_stations_icao = df_stations.select('neighbor_id', 'neighbor_call').withColumnRenamed("neighbor_call", "icao_station").distinct()

# Rename weather columns
df_weather = df_weather.withColumnRenamed("NAME", "STATION_NAME") \
  .withColumnRenamed("STATION", "STATION_ID") \
  .withColumnRenamed("ELEVATION", "STATION_ELEVATION") \
  .withColumnRenamed("LATITUDE", "STATION_LATITUDE") \
  .withColumnRenamed("LONGITUDE", "STATION_LONGITUDE") \
  .withColumnRenamed("DATE", "WEATHER_DATE") \
  .withColumnRenamed("REPORT_TYPE", "WEATHER_REPORT_TYPE") \
  .withColumnRenamed("SOURCE", "WEATHER_SOURCE")

columns_to_keep = ["HourlyDewPointTemperature", "HourlyDryBulbTemperature", "HourlyPrecipitation", "HourlyPresentWeatherType", "HourlyStationPressure", "HourlyVisibility", "HourlyWetBulbTemperature", "HourlyWindDirection", "HourlyWindSpeed", "YEAR"]
# Drop weather reports that are not FM-15 or 16
df_weather = df_weather.where("WEATHER_REPORT_TYPE in ('FM-15', 'FM-16')").select("STATION_ID", "STATION_NAME", "STATION_ELEVATION", "STATION_LATITUDE", "STATION_LONGITUDE", "WEATHER_DATE", "WEATHER_REPORT_TYPE", "WEATHER_SOURCE", *columns_to_keep)

# Add valid ICAO to weather data
df_weather_icao = df_stations_icao.join(df_weather,\
                                           df_weather.STATION_ID == df_stations_icao.neighbor_id,\
                                           'left').where('neighbor_id IS NOT NULL').drop("neighbor_id")

#Pull in timezone for weather stations
df_weather_icao_tz = df_weather_icao.join(df_airport_timezone,\
                                     df_weather_icao.icao_station == df_airport_timezone.icao,\
                                     'left').drop("icao_station")


# Merge airlines and timezones
flights_icao_tz = df_airlines.join(df_airport_timezone,\
                                     df_airlines.ORIGIN == df_airport_timezone.iata,\
                                     'left')

flights_icao_tz = flights_icao_tz.withColumn("flight_id", monotonically_increasing_id())


# Create unique origins/airports
origin_airports = df_airlines.select("ORIGIN").distinct().withColumnRenamed("ORIGIN", "AIRPORT_IATA")
dest_airports = df_airlines.select("DEST").distinct().withColumnRenamed("ORIGIN", "AIRPORT_IATA")
all_airports = origin_airports.union(dest_airports).distinct()

all_airports_icao = all_airports.join(df_airport_timezone, df_airport_timezone.iata == all_airports.AIRPORT_IATA).where("iata is not NULL").select("AIRPORT_IATA", "icao").withColumnRenamed("icao", "icao_flight")

# Drop entries not in flights data
df_weather_icao_needed_tz = df_weather_icao_tz.join(all_airports_icao,\
                                           df_weather_icao_tz.icao == all_airports_icao.icao_flight,\
                                           'left').where('AIRPORT_IATA IS NOT NULL').drop("AIRPORT_IATA", "icao_flight")

# Save the new datasets
flights_icao_tz.write.mode("overwrite").parquet(f"{blob_url}/flights_with_icao_tz")
df_weather_icao_needed_tz.write.mode("overwrite").parquet(f"{blob_url}/weather_with_icao_tz")

In [0]:
# Load the new datasets
flights_icao_tz = spark.read.parquet(f"{blob_url}/flights_with_icao_tz") # 42430592
df_weather_icao_needed_tz = spark.read.parquet(f"{blob_url}/weather_with_icao_tz")

In [0]:
from datetime import  datetime, timedelta
from pytz import timezone

def get_hour_minutes(depature_time):
    depature_time = str(depature_time)
    hour = ''
    if len(depature_time) <= 2:
      hour = '00'
      minutes = depature_time
    elif len(depature_time) == 3:
      hour = depature_time[0]
      minutes = depature_time[1:]
    elif len(depature_time) == 4:
      hour = depature_time[:2]
      minutes = depature_time[2:]
    return hour, minutes
  
def get_timestamp(year, month, day, hour_minutes, tz):
  hour, minutes = get_hour_minutes(hour_minutes)
  utc = timezone('UTC')
  tz = timezone(tz)
  timestamp = tz.localize(datetime(int(year), int(month), int(day), hour=int(hour), minute=int(minutes)))
  return timestamp.astimezone(utc)

def get_hour_timestamp(year, month, day, hour_minutes, tz):
  hour, minutes = get_hour_minutes(hour_minutes)
  utc = timezone('UTC')
  tz = timezone(tz)
  timestamp = tz.localize(datetime(int(year), int(month), int(day), hour=int(hour)))
  return timestamp.astimezone(utc)

def get_weather_timestamp(depature_timestamp, hours = 2):
  return (depature_timestamp - timedelta(hours=hours))

def go_back_an_hour(depature_timestamp):
  return (depature_timestamp - timedelta(hours=1))
 
def get_string_timestamp(timestamp):
  return timestamp.strftime("%Y%m%d%H%M")

def get_hourly_string_timestamp(timestamp):
  return timestamp.strftime("%Y%m%d%H")
 
get_timestamp = udf(get_timestamp)
get_hour_timestamp = udf(get_hour_timestamp)
get_weather_timestamp = udf(get_weather_timestamp)
get_string_timestamp = udf(get_string_timestamp)
get_hourly_string_timestamp = udf(get_hourly_string_timestamp)
go_back_an_hour = udf(go_back_an_hour)

# Create the timestamp columns for the flight data
flights_icao_tz = flights_icao_tz.withColumn("FLIGHT_TIMESTAMP", get_timestamp("YEAR", "MONTH", "DAY_OF_MONTH", "CRS_DEP_TIME", "airport_tz"))
flights_icao_tz = flights_icao_tz.withColumn("HOUR_WEATHER_TIMESTAMP", get_hour_timestamp("YEAR", "MONTH", "DAY_OF_MONTH", "CRS_DEP_TIME", "airport_tz"))
flights_icao_tz = flights_icao_tz.withColumn("WEATHER_TIMESTAMP", get_weather_timestamp("FLIGHT_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("HOUR_WEATHER_TIMESTAMP", get_weather_timestamp("HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("TWO_HOUR_WEATHER_TIMESTAMP", go_back_an_hour("HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("THREE_HOUR_WEATHER_TIMESTAMP", go_back_an_hour("TWO_HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("FLIGHT_TIMESTAMP", get_string_timestamp("FLIGHT_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("WEATHER_TIMESTAMP", get_string_timestamp("WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("HOUR_WEATHER_TIMESTAMP", get_hourly_string_timestamp("HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("TWO_HOUR_WEATHER_TIMESTAMP", get_hourly_string_timestamp("TWO_HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("THREE_HOUR_WEATHER_TIMESTAMP", get_hourly_string_timestamp("THREE_HOUR_WEATHER_TIMESTAMP"))
flights_icao_tz = flights_icao_tz.withColumn("FLIGHT_TIMESTAMP", to_timestamp(col("FLIGHT_TIMESTAMP"), "yyyyMMddHHmm"))
flights_icao_tz = flights_icao_tz.withColumn("WEATHER_TIMESTAMP", to_timestamp(col("WEATHER_TIMESTAMP"), "yyyyMMddHHmm"))

# Update the file with timestamps
flights_icao_tz.write.mode("overwrite").parquet(f"{blob_url}/flights_with_icao_tz")

In [0]:
def get_utc_difference(tz):
  utc_diff = {
    "America/Chicago": 6,
    "Pacific/Honolulu": 10,
    "America/Phoenix": 7,
    "America/Denver": 7,
    "America/Los_Angeles": 8,
    "America/New_York": 5,
    "America/Anchorage": 9,
    "America/Boise": 7,
    "America/Detroit": 5,
    "America/St_Thomas": 4,
    "America/Puerto_Rico": 4,
    "America/Adak": 10,
    "America/Juneau": 9,
    "America/Kentucky/Louisville": 5,
    "America/Indiana/Indianapolis": 5,
  }
  
  return utc_diff[tz] if tz in utc_diff else 0

def get_timestamp_weather_df(date, tz):
  if date is None or tz is None:
    return 'empty'
  diff = get_utc_difference(tz)
  utc = timezone('UTC')
  timestamp = utc.localize(datetime.fromisoformat(date)) + timedelta(hours=diff)
  return timestamp.strftime("%Y%m%d%H%M")

def get_hour_timestamp_weather_df(date, tz):
  if date is None or tz is None:
    return 'empty'
  diff = get_utc_difference(tz)
  utc = timezone('UTC')
  timestamp = utc.localize(datetime.fromisoformat(date))  + timedelta(hours=diff)
  return timestamp.strftime("%Y%m%d%H")

get_timestamp_weather_df = udf(get_timestamp_weather_df)
get_hour_timestamp_weather_df = udf(get_hour_timestamp_weather_df)
# Create the timestamp for the weather data
df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn("STATION_TIMESTAMP", get_timestamp_weather_df('WEATHER_DATE', 'airport_tz'))
df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn("HOUR_TIMESTAMP", get_hour_timestamp_weather_df('WEATHER_DATE', 'airport_tz'))
df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn("STATION_TIMESTAMP", to_timestamp(col("STATION_TIMESTAMP"), "yyyyMMddHHmm"))


In [0]:
#clean values oh hourly precipitation
def clean_hourly_precipitation(line, value= 0.0):
  if (line is None):
    value = 0.0
  elif ('T' in line) | (line == "") | ('*' in line):
    value = 0.0
  elif 's' in line:
    temp = line.replace('s', '')
    if len(line)>=8:
      value = (float(temp[:4])+float(temp[4:])) /2.0
    else:
      value = float(temp)
  elif len(line)>=8:
      value = (float(line[:4])+float(line[4:])) /2.0     
  else:
    value = float(line)
  return value  

clean_hourly_precipitation = udf(clean_hourly_precipitation)
# Create the timestamp for the weather data
df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn("HourlyPrecipitation", clean_hourly_precipitation('HourlyPrecipitation'))

In [0]:
# converte the hourly variables to numeric
from pyspark.sql.functions import col
columns_to_cast = ["HourlyDewPointTemperature","HourlyDryBulbTemperature","HourlyWetBulbTemperature", "HourlyStationPressure", "HourlyWindDirection", "HourlyWindSpeed", "HourlyVisibility"]
df_weather_icao_needed_tz = (
   df_weather_icao_needed_tz
   .select(
     *(c for c in df_weather_icao_needed_tz.columns if c not in columns_to_cast),
     *(col(c).cast("float").alias(c) for c in columns_to_cast)
   )
)

In [0]:
# Clean the Hourly weather event
from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, datediff, when, split

df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn('AU_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(0))\
                               .withColumn('AW_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(1))\
                               .withColumn('MW_code', split(col('HourlyPresentWeatherType'), '\\|').getItem(2))\

df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn('AU_TS', when( (col('AU_code').contains("+TS")) | (col('AU_code').contains("FC")),1).otherwise(0))\
                                   .withColumn('AU_ICE', when(col('AU_code').contains("IC" ), 1).otherwise(0))\
                                   .withColumn('AU_SNOW',when(col('AU_code').contains("+SN" ), 1).otherwise(0))\
                                   .withColumn('AU_FOG', when(col('AU_code').contains("FG" ), 1).otherwise(0))

df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn('AW_TS', when((col('AW_code').contains("TS" ))| (col('AW_code').contains("FC")) , 1).otherwise(0))\
                                   .withColumn('AW_ICE', when(col('AW_code').contains("FZRA" ), 1).otherwise(0))\
                                   .withColumn('AW_SNOW',when(col('AW_code').contains("SN"), 1).otherwise(0))\
                                   .withColumn('AW_FOG', when(col('AW_code').contains("FG"), 1).otherwise(0))

df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn('MW_TS', when((col('MW_code').contains("TS" ))| (col('MW_code').contains("FC")), 1).otherwise(0))\
                                  .withColumn('MW_ICE', when(col('MW_code').contains("FZRA"), 1).otherwise(0))\
                                  .withColumn('MW_SNOW',when(col('MW_code').contains("SN" ), 1).otherwise(0))\
                                  .withColumn('MW_FOG', when(col('MW_code').contains("FG"), 1).otherwise(0))

df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn('TS',when( (col('AU_TS')== 1) | (col('AW_TS')== 1)| (col('MW_TS')== 1), 1).otherwise(0))\
                                  .withColumn('ICE', when((col('AU_ICE')== 1 )| (col('AW_ICE')== 1)| (col('MW_ICE')== 1), 1).otherwise(0))\
                                  .withColumn('SNOW',when((col('AU_SNOW')== 1 )| (col('AW_SNOW')== 1)| (col('MW_SNOW')== 1), 1).otherwise(0))\
                                  .withColumn('FOG', when((col('AU_FOG')== 1 )| (col('AW_FOG')== 1)| (col('MW_FOG')== 1), 1).otherwise(0))\


# Update weather dataset
df_weather_icao_needed_tz.write.mode("overwrite").parquet(f"{blob_url}/weather_with_icao_tz")

In [0]:
# Load the new datasets
flights_icao_tz = spark.read.parquet(f"{blob_url}/flights_with_icao_tz") # 42430592
df_weather_icao_needed_tz = spark.read.parquet(f"{blob_url}/weather_with_icao_tz") # 31717569

42430592
31717569


In [0]:
def create_composite_key(code, timestamp):
  return f'{code}_{timestamp}'

create_composite_key = udf(create_composite_key)

# Create composite key for both datasets
df_weather_icao_needed_tz = df_weather_icao_needed_tz.withColumn("CODE_STATION_TIMESTAMP", create_composite_key('icao', 'HOUR_TIMESTAMP'))
flights_icao_tz = flights_icao_tz.withColumn("CODE_TIMESTAMP", create_composite_key('icao', 'HOUR_WEATHER_TIMESTAMP')) \
  .withColumn("TWO_CODE_TIMESTAMP", create_composite_key('icao', 'TWO_HOUR_WEATHER_TIMESTAMP')) \
  .withColumn("THREE_CODE_TIMESTAMP", create_composite_key('icao', 'THREE_HOUR_WEATHER_TIMESTAMP'))

# Update the datasets
flights_icao_tz.write.mode("overwrite").parquet(f"{blob_url}/flights_with_icao_tz")
df_weather_icao_needed_tz.write.mode("overwrite").parquet(f"{blob_url}/weather_with_icao_tz")

In [0]:
# Load the new datasets
flights_icao_tz = spark.read.parquet(f"{blob_url}/flights_with_icao_tz") # 42430592
df_weather_icao_needed_tz = spark.read.parquet(f"{blob_url}/weather_with_icao_tz") # 1057832
print(flights_icao_tz.count())

42430592


In [0]:
# Drop unnecessary columns
columns_to_drop = ['airport_name', 'airport_city', 'airport_country', 'airport_tz', 'year', 'airport_subd', 'country', 'elevation', 'iata', 'airport_lon', 'airport_lat', 'icao']
df_weather_icao_needed_tz = df_weather_icao_needed_tz.drop(*columns_to_drop)

# Join the datasets
joined_df_one = flights_icao_tz.join(df_weather_icao_needed_tz, flights_icao_tz.CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')
joined_df_two = flights_icao_tz.join(df_weather_icao_needed_tz, flights_icao_tz.TWO_CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')
joined_df_three = flights_icao_tz.join(df_weather_icao_needed_tz, flights_icao_tz.THREE_CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')

combined_df = joined_df_one.union(joined_df_two).union(joined_df_three) # 3m 6204054
combined_df.write.mode("overwrite").parquet(f"{blob_url}/combined")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1013197717890732> in <cell line: 11>()
      9 
     10 combined_df = joined_df_one.union(joined_df_two).union(joined_df_three) # 3m 6204054
---> 11 combined_df.write.mode("overwrite").parquet(f"{blob_url}/combined")

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in parquet(self, path, mode, partitionBy, compression)
   1138             self.partitionBy(partitionBy)
   1139         self._set_opts(compression=compression)
-> 1140         self._jwrite.parquet(path)
   

In [0]:
combined_df = spark.read.parquet(f"{blob_url}/combined") # 184162262
combined_df.count()

Out[42]: 156764783

In [0]:
from pyspark.sql.functions import min

joined_df_pos = combined_df.withColumn("distance_time",col("WEATHER_TIMESTAMP").cast("long") - col("STATION_TIMESTAMP").cast("long")).where("distance_time >= 0")
joined_df_pos_min = joined_df_pos.groupBy("flight_id").agg(min("distance_time").alias('minimum_distance')).withColumnRenamed("flight_id", "flight_id_min")
joined_df_pos_min_final = joined_df_pos.join(joined_df_pos_min, (joined_df_pos_min.flight_id_min == joined_df_pos.flight_id) & (joined_df_pos_min.minimum_distance == joined_df_pos.distance_time), 'inner') # 3m 1396256 unique

joined_df_clean = joined_df_pos_min_final.dropDuplicates(["flight_id"])
joined_df_clean.write.mode("overwrite").parquet(f"{blob_url}/joined_df_full")

In [0]:
# Drop temporary columns
columns_to_drop = ['CODE_STATION_TIMESTAMP', 'distance_time', 'flight_id_min', 'minimum_distance', 'CODE_TIMESTAMP', 'TWO_CODE_TIMESTAMP', 'THREE_CODE_TIMESTAMP']

clean_full = spark.read.parquet(f"{blob_url}/joined_df_full") # 42227239
clean_full = clean_full.drop(*columns_to_drop)

# Rename the weather features so it's clear they belong to the ORIGIN airport
columns_to_rename = ['STATION_ID', 'STATION_NAME', 'STATION_ELEVATION', 'STATION_LATITUDE', 'STATION_LONGITUDE', 'WEATHER_DATE', 'WEATHER_REPORT_TYPE', 'WEATHER_SOURCE', 'HourlyPrecipitation', 'HourlyPresentWeatherType', 'STATION_TIMESTAMP', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyWetBulbTemperature', 'HourlyStationPressure', 'HourlyWindDirection', 'HourlyWindSpeed', 'HourlyVisibility', 'AU_code', 'AW_code', 'MW_code', 'AU_TS', 'AU_ICE', 'AU_SNOW', 'AU_FOG', 'AW_TS', 'AW_ICE', 'AW_SNOW', 'AW_FOG', 'MW_TS', 'MW_ICE', 'MW_SNOW', 'MW_FOG', 'TS', 'ICE', 'SNOW', 'FOG', 'icao', 'iata', 'airport_name', 'airport_city', 'airport_subd', 'airport_country', 'elevation', 'airport_lat', 'airport_lon', 'airport_tz', 'FLIGHT_TIMESTAMP']


for column in columns_to_rename:
  clean_full = clean_full.withColumnRenamed(column, "ORIGIN_" + column)

clean_full.write.mode("overwrite").parquet(f"{blob_url}/joined_df_full")

In [0]:
clean_full = spark.read.parquet(f"{blob_url}/joined_df_full") # 42227239
origin_airports = clean_full.select("ORIGIN").distinct()
dest_airports = clean_full.select("DEST").distinct()

joined = dest_airports.join(origin_airports, origin_airports.ORIGIN == dest_airports.DEST, 'left').where("ORIGIN is null")
display(joined)

DEST,ORIGIN
PSE,null
PPG,null
OGS,null
SJU,null
GUM,null
XWA,null
FNL,null


In [0]:
clean_full = spark.read.parquet(f"{blob_url}/joined_df_full") # 42227239
full_dest_tz = clean_full.join(df_airport_timezone, clean_full.DEST == df_airport_timezone.iata, 'left')

full_dest_tz = full_dest_tz.withColumn("CODE_TIMESTAMP", create_composite_key('icao', 'HOUR_WEATHER_TIMESTAMP')) \
  .withColumn("TWO_CODE_TIMESTAMP", create_composite_key('icao', 'TWO_HOUR_WEATHER_TIMESTAMP')) \
  .withColumn("THREE_CODE_TIMESTAMP", create_composite_key('icao', 'THREE_HOUR_WEATHER_TIMESTAMP'))

# Drop unnecessary columns
df_weather_icao_needed_tz = spark.read.parquet(f"{blob_url}/weather_with_icao_tz") # 1057832
columns_to_drop = ['airport_name', 'airport_city', 'airport_country', 'airport_tz', 'year', 'airport_subd', 'country', 'elevation', 'iata', 'airport_lon', 'airport_lat', 'icao', 'HOUR_TIMESTAMP']
df_weather_icao_needed_tz = df_weather_icao_needed_tz.drop(*columns_to_drop)

# Join the datasets
joined_df_one = full_dest_tz.join(df_weather_icao_needed_tz, full_dest_tz.CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')
joined_df_two = full_dest_tz.join(df_weather_icao_needed_tz, full_dest_tz.TWO_CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')
joined_df_three = full_dest_tz.join(df_weather_icao_needed_tz, full_dest_tz.THREE_CODE_TIMESTAMP == df_weather_icao_needed_tz.CODE_STATION_TIMESTAMP, 'left')

combined_df = joined_df_one.union(joined_df_two).union(joined_df_three) # 3m 6204054
combined_df.write.mode("overwrite").parquet(f"{blob_url}/combined_dest")


In [0]:
combined_df = spark.read.parquet(f"{blob_url}/combined_dest") # 184162262

from pyspark.sql.functions import min

joined_df_pos = combined_df.withColumn("distance_time",col("WEATHER_TIMESTAMP").cast("long") - col("STATION_TIMESTAMP").cast("long")).where("distance_time >= 0")
joined_df_pos_min = joined_df_pos.groupBy("flight_id").agg(min("distance_time").alias('minimum_distance')).withColumnRenamed("flight_id", "flight_id_min")
joined_df_pos_min_final = joined_df_pos.join(joined_df_pos_min, (joined_df_pos_min.flight_id_min == joined_df_pos.flight_id) & (joined_df_pos_min.minimum_distance == joined_df_pos.distance_time), 'inner') # 3m 1396256 unique

joined_df_clean = joined_df_pos_min_final.dropDuplicates(["flight_id"])
joined_df_clean.write.mode("overwrite").parquet(f"{blob_url}/joined_df_full_dest")

In [0]:
# Drop temporary columns
columns_to_drop = ['CODE_STATION_TIMESTAMP', 'distance_time', 'flight_id_min', 'minimum_distance', 'CODE_TIMESTAMP', 'TWO_CODE_TIMESTAMP', 'THREE_CODE_TIMESTAMP']

join_df_origin_dest = spark.read.parquet(f"{blob_url}/joined_df_full_dest") # 184162262
join_df_origin_dest = join_df_origin_dest.drop(*columns_to_drop)

# Rename the weather features so it's clear they belong to the ORIGIN airport
columns_to_rename = ['STATION_ID', 'STATION_NAME', 'STATION_ELEVATION', 'STATION_LATITUDE', 'STATION_LONGITUDE', 'WEATHER_DATE', 'WEATHER_REPORT_TYPE', 'WEATHER_SOURCE', 'HourlyPrecipitation', 'HourlyPresentWeatherType', 'STATION_TIMESTAMP', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyWetBulbTemperature', 'HourlyStationPressure', 'HourlyWindDirection', 'HourlyWindSpeed', 'HourlyVisibility', 'AU_code', 'AW_code', 'MW_code', 'AU_TS', 'AU_ICE', 'AU_SNOW', 'AU_FOG', 'AW_TS', 'AW_ICE', 'AW_SNOW', 'AW_FOG', 'MW_TS', 'MW_ICE', 'MW_SNOW', 'MW_FOG', 'TS', 'ICE', 'SNOW', 'FOG', 'icao', 'iata', 'airport_name', 'airport_city', 'airport_subd', 'airport_country', 'elevation', 'airport_lat', 'airport_lon', 'airport_tz', 'FLIGHT_TIMESTAMP']


for column in columns_to_rename:
  join_df_origin_dest = join_df_origin_dest.withColumnRenamed(column, "DEST_" + column)

join_df_origin_dest.write.mode("overwrite").parquet(f"{blob_url}/joined_df_full_dest")

In [0]:
join_df_origin_dest = spark.read.parquet(f"{blob_url}/joined_df_full_dest") # 184162262
display(join_df_origin_dest)

QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,YEAR,ORIGIN_icao,ORIGIN_iata,ORIGIN_airport_name,ORIGIN_airport_city,ORIGIN_airport_subd,ORIGIN_airport_country,ORIGIN_elevation,ORIGIN_airport_lat,ORIGIN_airport_lon,ORIGIN_airport_tz,flight_id,ORIGIN_FLIGHT_TIMESTAMP,HOUR_WEATHER_TIMESTAMP,WEATHER_TIMESTAMP,TWO_HOUR_WEATHER_TIMESTAMP,THREE_HOUR_WEATHER_TIMESTAMP,ORIGIN_STATION_ID,ORIGIN_STATION_NAME,ORIGIN_STATION_ELEVATION,ORIGIN_STATION_LATITUDE,ORIGIN_STATION_LONGITUDE,ORIGIN_WEATHER_DATE,ORIGIN_WEATHER_REPORT_TYPE,ORIGIN_WEATHER_SOURCE,ORIGIN_HourlyPrecipitation,ORIGIN_HourlyPresentWeatherType,ORIGIN_STATION_TIMESTAMP,HOUR_TIMESTAMP,ORIGIN_HourlyDewPointTemperature,ORIGIN_HourlyDryBulbTemperature,ORIGIN_HourlyWetBulbTemperature,ORIGIN_HourlyStationPressure,ORIGIN_HourlyWindDirection,ORIGIN_HourlyWindSpeed,ORIGIN_HourlyVisibility,ORIGIN_AU_code,ORIGIN_AW_code,ORIGIN_MW_code,ORIGIN_AU_TS,ORIGIN_AU_ICE,ORIGIN_AU_SNOW,ORIGIN_AU_FOG,ORIGIN_AW_TS,ORIGIN_AW_ICE,ORIGIN_AW_SNOW,ORIGIN_AW_FOG,ORIGIN_MW_TS,ORIGIN_MW_ICE,ORIGIN_MW_SNOW,ORIGIN_MW_FOG,ORIGIN_TS,ORIGIN_ICE,ORIGIN_SNOW,ORIGIN_FOG,DEST_icao,DEST_iata,DEST_airport_name,DEST_airport_city,DEST_airport_subd,DEST_airport_country,DEST_elevation,DEST_airport_lat,DEST_airport_lon,DEST_airport_tz,DEST_STATION_ID,DEST_STATION_NAME,DEST_STATION_ELEVATION,DEST_STATION_LATITUDE,DEST_STATION_LONGITUDE,DEST_WEATHER_DATE,DEST_WEATHER_REPORT_TYPE,DEST_WEATHER_SOURCE,DEST_HourlyPrecipitation,DEST_HourlyPresentWeatherType,DEST_STATION_TIMESTAMP,DEST_HourlyDewPointTemperature,DEST_HourlyDryBulbTemperature,DEST_HourlyWetBulbTemperature,DEST_HourlyStationPressure,DEST_HourlyWindDirection,DEST_HourlyWindSpeed,DEST_HourlyVisibility,DEST_AU_code,DEST_AW_code,DEST_MW_code,DEST_AU_TS,DEST_AU_ICE,DEST_AU_SNOW,DEST_AU_FOG,DEST_AW_TS,DEST_AW_ICE,DEST_AW_SNOW,DEST_AW_FOG,DEST_MW_TS,DEST_MW_ICE,DEST_MW_SNOW,DEST_MW_FOG,DEST_TS,DEST_ICE,DEST_SNOW,DEST_FOG
4,12,4,1,2017-12-04,UA,19977,UA,N77537,1688,14771,1477104,32457,SFO,"San Francisco, CA",CA,6,California,91,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,1715,1708,-7.0,0.0,0.0,-1,1700-1759,17.0,1725,1908,8.0,1927,1916,-11.0,0.0,0.0,-1,1900-1959,0.0,null,0.0,132.0,128.0,103.0,1.0,679.0,3,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [0]:
df_airlines.where("ORIGIN in ('PSE', 'PPG', 'OGS', 'SPN', 'SJU', 'GUM', 'XWA')").count()

Out[16]: 193037

In [0]:
from pyspark.sql.functions import countDistinct
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Find missing airports

matched_airpoorts = clean_full.select("ORIGIN").withColumnRenamed("ORIGIN", "MATCHED_ORIGIN").distinct()

original_airports = df_airlines.select("ORIGIN").distinct()

missing_airports = original_airports.join(matched_airpoorts, original_airports.ORIGIN == matched_airpoorts.MATCHED_ORIGIN, "left").where("MATCHED_ORIGIN is NULL")

# Find neirest stations to each airport
just_stations = df_stations.select("station_id", "lon", "lat").distinct()

# Get airport data for missing airports
missing_airports_coords = missing_airports.join(df_airport_timezone, missing_airports.ORIGIN == df_airport_timezone.iata, "left")

def haversine_distance(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    d_lon = lon2 - lon1 
    d_lat = lat2 - lat1 
    a = sin(d_lat/2)**2 + cos(lat1) * cos(lat2) * sin(d_lon/2)**2
    c = 2 * asin(sqrt(a)) 
    rkm = 6371
    rm = 3956
    return c * rkm, c * rm

stations_sc = sc.broadcast(just_stations.rdd.collect())
    
def calc_distances(airport):
  airport_lon = float(airport['airport_lon'])
  airport_lat = float(airport['airport_lat'])

  for station in stations_sc.value:
    if not station['lon'] or not station['lat']:
        continue
    station_lon = float(station['lon'])
    station_lat = float(station['lat'])
    station_id = station['station_id']
    yield (airport['iata'], (station_id, haversine_distance(airport_lon, airport_lat, station_lon, station_lat)))

def find_minimum(a, b):
  minimum_index = np.argmin([a[1][0], b[1][0]])
  if minimum_index == 0:
      return a
  else:
      return b
    
def map_values(a):
  return (a[0], a[1][0], a[1][1][0], a[1][1][1])
result = missing_airports_coords.rdd.flatMap(calc_distances).reduceByKey(find_minimum).map(map_values).collect()
new_stations = spark.createDataFrame(result, ["IATA", "CLOSEST_STATION_ID", "DISTANCE in KM", "DISTANCE in Miles"])
new_stations.show()

+----+------------------+------------------+------------------+
|IATA|CLOSEST_STATION_ID|    DISTANCE in KM| DISTANCE in Miles|
+----+------------------+------------------+------------------+
| GUM|       70414045715| 5074.661619495768| 3151.053424380044|
| XWA|       72767094014|12.720358338679711| 7.898561856508701|
| PPG|       91197521510|4113.6394938583635| 2554.317664056457|
| SPN|       70414045715| 4867.467580900914| 3022.398642292264|
| OGS|       72622394725| 56.45093250281826|35.052564586587515|
| SJU|       78526500494|10.334183951503903| 6.416894005987983|
| PSE|       78514511653| 67.68333842958074| 42.02719931367468|
+----+------------------+------------------+------------------+

